# Thesis

#### Importing Data From the '19-'20 Season

The first thing we need to do is import all of our necessary libraries. 

In [112]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk


Reading the dataset and importing it into pandas as a DataFrame

In [113]:
PBP20 = pd.read_csv('/Users/seandiehl/Desktop/NBA Play By Play Data/NBA_PBP_2019-20.csv')
pd.set_option('display.max_columns', None)

Here, we are simply inspecting the DataFrame to ensure it looks as expected based on the csv file.

In [114]:
PBP20.head()

,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,AwayScore,HomeTeam,HomePlay,HomeScore,Shooter,ShotType,ShotOutcome,ShotDist,Assister,Blocker,FoulType,Fouler,Fouled,Rebounder,ReboundType,ViolationPlayer,ViolationType,TimeoutTeam,FreeThrowShooter,FreeThrowOutcome,FreeThrowNum,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss,Unnamed: 40
0,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,720,NOP,Jump ball: D. Favors vs. M. Gasol (L. Ball gai...,0,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D. Favors - favorde01,M. Gasol - gasolma01,L. Ball - balllo01,NaN
1,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,708,NOP,L. Ball misses 2-pt jump shot from 11 ft,0,TOR,NaN,0,L. Ball - balllo01,2-pt jump shot,miss,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,Offensive rebound by D. Favors,0,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D. Favors - favorde01,offensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,D. Favors makes 2-pt layup at rim,2,TOR,NaN,0,D. Favors - favorde01,2-pt layup,make,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,689,NOP,NaN,2,TOR,O. Anunoby misses 2-pt layup from 3 ft,0,O. Anunoby - anunoog01,2-pt layup,miss,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
PBP20.rename({'Unnamed: 40': 'a'}, axis='columns', inplace=True)
PBP20.drop(['a'], axis=1, inplace=True)

PBP20.head()

,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,AwayScore,HomeTeam,HomePlay,HomeScore,Shooter,ShotType,ShotOutcome,ShotDist,Assister,Blocker,FoulType,Fouler,Fouled,Rebounder,ReboundType,ViolationPlayer,ViolationType,TimeoutTeam,FreeThrowShooter,FreeThrowOutcome,FreeThrowNum,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss
0,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,720,NOP,Jump ball: D. Favors vs. M. Gasol (L. Ball gai...,0,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D. Favors - favorde01,M. Gasol - gasolma01,L. Ball - balllo01
1,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,708,NOP,L. Ball misses 2-pt jump shot from 11 ft,0,TOR,NaN,0,L. Ball - balllo01,2-pt jump shot,miss,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,Offensive rebound by D. Favors,0,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D. Favors - favorde01,offensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,D. Favors makes 2-pt layup at rim,2,TOR,NaN,0,D. Favors - favorde01,2-pt layup,make,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,689,NOP,NaN,2,TOR,O. Anunoby misses 2-pt layup from 3 ft,0,O. Anunoby - anunoog01,2-pt layup,miss,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Next, for the relevant columns, stripping the player's Name from the data so that they are left with solely the PlayerID - just makes searching, indexing, etc., a little bit easier down the line.

In [116]:
PBP20['Shooter'] = PBP20['Shooter'].str.split('- ').str[1]
PBP20['Assister'] = PBP20['Assister'].str.split('- ').str[1]
PBP20['Blocker'] = PBP20['Blocker'].str.split('- ').str[1]
PBP20['Fouler'] = PBP20['Fouler'].str.split('- ').str[1]
PBP20['Fouled'] = PBP20['Fouled'].str.split('- ').str[1]
PBP20['Rebounder'] = PBP20['Rebounder'].str.split('- ').str[1]
PBP20['ViolationPlayer'] = PBP20['ViolationPlayer'].str.split('- ').str[1]
PBP20['FreeThrowShooter'] = PBP20['FreeThrowShooter'].str.split('- ').str[1]
PBP20['EnterGame'] = PBP20['EnterGame'].str.split('- ').str[1]
PBP20['LeaveGame'] = PBP20['LeaveGame'].str.split('- ').str[1]
PBP20['TurnoverPlayer'] = PBP20['TurnoverPlayer'].str.split('- ').str[1]
PBP20['TurnoverCauser'] = PBP20['TurnoverCauser'].str.split('- ').str[1]
PBP20['JumpballAwayPlayer'] = PBP20['JumpballAwayPlayer'].str.split('-').str[1]
PBP20['JumpballHomePlayer'] = PBP20['JumpballHomePlayer'].str.split('-').str[1]
PBP20['JumpballPoss'] = PBP20['JumpballPoss'].str.split('-').str[1]

Next, we are going to create a new column, a Unique GameID for each game played throughout the season. <br>

As it stands to reason that each game will have its own unique URL link associated with it, we can simply tell pandas to add 1 to the GameID value each time it comes across a new URL. <br>

Again, this will help with indexing later. 

In [117]:
PBP20['GameID'] = PBP20.groupby('URL').ngroup()


In [118]:
PBP20.head()

,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,AwayScore,HomeTeam,HomePlay,HomeScore,Shooter,ShotType,ShotOutcome,ShotDist,Assister,Blocker,FoulType,Fouler,Fouled,Rebounder,ReboundType,ViolationPlayer,ViolationType,TimeoutTeam,FreeThrowShooter,FreeThrowOutcome,FreeThrowNum,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss,GameID
0,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,720,NOP,Jump ball: D. Favors vs. M. Gasol (L. Ball gai...,0,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,favorde01,gasolma01,balllo01,1
1,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,708,NOP,L. Ball misses 2-pt jump shot from 11 ft,0,TOR,NaN,0,balllo01,2-pt jump shot,miss,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,Offensive rebound by D. Favors,0,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,favorde01,offensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,D. Favors makes 2-pt layup at rim,2,TOR,NaN,0,favorde01,2-pt layup,make,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,689,NOP,NaN,2,TOR,O. Anunoby misses 2-pt layup from 3 ft,0,anunoog01,2-pt layup,miss,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


### Seconds Passed in Game Column

Here, we are going to add a column that represents the total seconds passed in the game. We have a column with the value for each quarter, and we have a column with the value for each 

In [119]:
conditions = [
    (PBP20['Quarter'] == 1), (PBP20['Quarter'] == 2), (PBP20['Quarter']==3), (PBP20['Quarter'] ==4), (PBP20['Quarter']==5), (PBP20['Quarter']==6)
]

values = [0, 720, 1440, 2160, 2880, 3180]

PBP20['SecStartQuarter'] = np.select(conditions, values)

PBP20['SecPassed'] = PBP20['SecStartQuarter'] + (720 - PBP20['SecLeft'])

In [120]:
PBP20.head()

,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,AwayScore,HomeTeam,HomePlay,HomeScore,Shooter,ShotType,ShotOutcome,ShotDist,Assister,Blocker,FoulType,Fouler,Fouled,Rebounder,ReboundType,ViolationPlayer,ViolationType,TimeoutTeam,FreeThrowShooter,FreeThrowOutcome,FreeThrowNum,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss,GameID,SecStartQuarter,SecPassed
0,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,720,NOP,Jump ball: D. Favors vs. M. Gasol (L. Ball gai...,0,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,favorde01,gasolma01,balllo01,1,0,0
1,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,708,NOP,L. Ball misses 2-pt jump shot from 11 ft,0,TOR,NaN,0,balllo01,2-pt jump shot,miss,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,12
2,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,Offensive rebound by D. Favors,0,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,favorde01,offensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,13
3,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,D. Favors makes 2-pt layup at rim,2,TOR,NaN,0,favorde01,2-pt layup,make,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,13
4,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,689,NOP,NaN,2,TOR,O. Anunoby misses 2-pt layup from 3 ft,0,anunoog01,2-pt layup,miss,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,31


### Lineups - Who's on the court when? 
Next thing we need to do is figure out who is on the court at what times. Because we are working expected average statistics that are adjusted based on the time spent on the court, we need an accurate value for how long, and at what times, each player was on the court during each game.

Workflow: 

1 - create new table that holds the enter game and leave game stats 


In [121]:
subs_df = PBP20[['Quarter', 'SecLeft', 'SecPassed', 'EnterGame', 'LeaveGame', 'GameID']]

subs_df.head()

,Quarter,SecLeft,SecPassed,EnterGame,LeaveGame,GameID
0,1,720,0,NaN,NaN,1
1,1,708,12,NaN,NaN,1
2,1,707,13,NaN,NaN,1
3,1,707,13,NaN,NaN,1
4,1,689,31,NaN,NaN,1


In [122]:
subs_df = subs_df.dropna(subset=['EnterGame'])

In [123]:
subs_df.head()

,Quarter,SecLeft,SecPassed,EnterGame,LeaveGame,GameID
47,1,396,324,ibakase01,gasolma01,1
61,1,320,400,powelno01,lowryky01,1
66,1,286,434,hartjo01,balllo01,1
67,1,286,434,mooreet01,favorde01,1
68,1,286,434,okafoja01,redicjj01,1


In [124]:
subs_df.groupby(['GameID', 'Quarter', 'EnterGame']).head()

,Quarter,SecLeft,SecPassed,EnterGame,LeaveGame,GameID
47,1,396,324,ibakase01,gasolma01,1
61,1,320,400,powelno01,lowryky01,1
66,1,286,434,hartjo01,balllo01,1
67,1,286,434,mooreet01,favorde01,1
68,1,286,434,okafoja01,redicjj01,1
...,...,...,...,...,...,...
539240,4,144,2736,nunnke01,butleji01,1142
539248,4,87,2793,cookqu01,davisan02,1142
539249,4,87,2793,dudleja01,jamesle01,1142
539250,4,87,2793,jonesde02,robindu01,1142


#First, we want to filter out the subs column

In [126]:
participated_in_event = PBP20[PBP20['EnterGame'].isnull()]

,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,AwayScore,HomeTeam,HomePlay,HomeScore,Shooter,ShotType,ShotOutcome,ShotDist,Assister,Blocker,FoulType,Fouler,Fouled,Rebounder,ReboundType,ViolationPlayer,ViolationType,TimeoutTeam,FreeThrowShooter,FreeThrowOutcome,FreeThrowNum,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss,GameID,SecStartQuarter,SecPassed
0,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,720,NOP,Jump ball: D. Favors vs. M. Gasol (L. Ball gai...,0,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,favorde01,gasolma01,balllo01,1,0,0
1,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,708,NOP,L. Ball misses 2-pt jump shot from 11 ft,0,TOR,NaN,0,balllo01,2-pt jump shot,miss,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,12
2,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,Offensive rebound by D. Favors,0,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,favorde01,offensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,13
3,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,D. Favors makes 2-pt layup at rim,2,TOR,NaN,0,favorde01,2-pt layup,make,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,13
4,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,689,NOP,NaN,2,TOR,O. Anunoby misses 2-pt layup from 3 ft,0,anunoog01,2-pt layup,miss,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,31
5,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,685,NOP,Defensive rebound by J. Redick,2,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,redicjj01,defensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,35
6,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,676,NOP,J. Holiday misses 2-pt layup from 8 ft,2,TOR,NaN,0,holidjr01,2-pt layup,miss,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,44
7,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,675,NOP,NaN,2,TOR,Defensive rebound by F. VanVleet,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vanvlfr01,defensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,45
8,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,671,NOP,NaN,2,TOR,K. Lowry misses 3-pt jump shot from 25 ft,0,lowryky01,3-pt jump shot,miss,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,49
9,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,668,NOP,Defensive rebound by D. Favors,2,TOR,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,favorde01,defensive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,52
